In [1]:
import os
import datacube
import numpy as np
import xarray as xr
import subprocess as sp
import geopandas as gpd
from odc.io.cgroups import get_cpu_quota
from datacube.utils.geometry import assign_crs
import multiprocessing as mp

import sys
sys.path.insert(1, '../../Tools/')
from dea_tools.bandindices import calculate_indices
from dea_tools.classification import collect_training_data

import warnings
warnings.filterwarnings("ignore")

In [2]:
path = '../../clipped shape file/clipped Geojson.geojson'
field = 'PXLVAL'

In [3]:
ncpus = round(mp.cpu_count())
print('ncpus = '+str(ncpus))

ncpus = 12


In [4]:
# Load input data shapefile
input_data = gpd.read_file(path)

# Plot first five rows
input_data.head()

,PXLVAL,geometry
0,1,"MULTIPOLYGON (((81.10978 16.00000, 81.11022 16..."
1,1,"MULTIPOLYGON (((81.13356 16.00000, 81.13422 16..."
2,1,"MULTIPOLYGON (((81.13511 15.99933, 81.13533 15..."
3,1,"MULTIPOLYGON (((81.13489 16.00000, 81.13844 16..."
4,1,"MULTIPOLYGON (((81.12622 16.00000, 81.12956 16..."


In [5]:
# Plot training data in an interactive map
input_data.explore(column=field)


In [6]:
# Set up our inputs to collect_training_data


In [8]:
# Generate a new datacube query object
lat_range = (15.75418332, 15.85828652)
lon_range = (80.78694696, 81.02203692)
# path = '../clipped shape file/clipped_mangroves.shp'
# field = 'PXLVAL'
products = 's2a_sen2cor_granule'
measurements =  ['blue','green','red','nir','swir_1','swir_2']
time = ('2023-03-15', '2023-05-01')
zonal_stats = None #'median'
resolution =  (-30, 30)
query = {
    'x': lon_range,
    'y': lat_range,
    'time': time,
    'measurements': measurements,
    'resolution': resolution,
    'output_crs' : 'epsg:6933'
}

In [9]:
def feature_function(query):
    dc = datacube.Datacube(app='feature_layers')
    ds = dc.load(**query)
    ds = ds.mean('time')
    return ds

In [10]:
def feature_layers(query):
    #connect to the datacube
    dc = datacube.Datacube(app='feature_layers')

    #load ls8 geomedian
    
    ds = dc.load(product="s2a_sen2cor_granule",
                 **query)

    #calculate some band indices
    ds = calculate_indices(ds,
                           index=['NDVI', 'BUI', 'MNDWI'],
                           drop=False,
                           satellite_mission='s2')
    return ds

In [13]:
column_names, model_input = collect_training_data(
                                    gdf=input_data,
                                    dc_query=query,
                                    ncpus=ncpus,
                                    field=field,
                                    zonal_stats=zonal_stats,
                                    feature_func=feature_layers
                                    )

  0%|          | 0/2153 [00:00<?, ?it/s]

IndexError: list index out of range